In [1]:
import pandas as pd

In [2]:
shipin = pd.read_csv('Data/Table of Shipin.csv')
shipin

,Dynasty,Author,Rank,Origin,Origin Full,Remarks on Literature,Additional Remarks
0,NaN,《古诗》,上,《国风》,其体源出于《国风》。,NaN,NaN
1,汉,李陵,上,《楚辞》,其源出于《楚辞》。,文多凄怆，怨者之流。,陵，名家子，有殊才，生命不谐，声颓身丧。使陵不遭辛苦，其文亦何能至此！
2,汉,班姬,上,李陵,其源出于李陵。,《团扇》短章，辞旨清捷，怨深文绮，得匹妇之致。侏儒一节，可以知其工矣。,NaN
3,魏,曹植,上,《国风》,其源出于《国风》。,骨气奇高，词彩华茂，情兼雅怨，体被文质，粲溢今古，卓尔不群。嗟乎！陈思之于文章也，譬人伦之有...,NaN
4,魏,刘桢,上,《古诗》,其源出于《古诗》。,仗气爱奇，动多振绝。贞骨凌霜，高风跨俗。但气过其文，雕润恨少。然自陈思已下，桢称独步。,NaN
...,...,...,...,...,...,...,...
117,齐,陆厥,下,NaN,NaN,自制未优，非言之失也。,观厥文纬，具识文之情状。
118,梁,虞羲,下,NaN,NaN,子阳诗奇句清拔，谢朓常嗟颂之。,NaN
119,梁,江洪,下,NaN,NaN,洪虽无多，亦能自迥出。,NaN
120,梁,鲍行卿,下,NaN,NaN,行卿少年，甚擅风谣之美。,NaN


In [3]:
from collections import defaultdict
graph = defaultdict(list)
multiple_origin_flags = set()

for _, row in shipin.iterrows():
    author = row["Author"]
    origins = row["Origin"]
    if pd.notna(origins) and origins.strip():
        origin_list = [o.strip() for o in origins.split("、")]
        if len(origin_list) > 1:
            multiple_origin_flags.add(author)
        for origin in origin_list:
            graph[origin].append(author)
graph

defaultdict(list,
            {'《国风》': ['《古诗》', '曹植'],
             '《楚辞》': ['李陵'],
             '李陵': ['班姬', '王粲', '曹丕'],
             '《古诗》': ['刘桢'],
             '《小雅》': ['阮籍'],
             '曹植': ['陆机', '谢灵运'],
             '王粲': ['潘岳', '张协', '曹丕', '张华', '刘琨', '卢谌'],
             '刘桢': ['左思'],
             '张协': ['谢灵运', '鲍照'],
             '曹丕': ['嵇康', '应璩'],
             '潘岳': ['郭璞'],
             '应璩': ['陶潜'],
             '左思': ['陶潜'],
             '陆机': ['颜延之'],
             '张华': ['谢瞻', '谢混', '袁淑', '王微', '王僧达', '鲍照'],
             '谢混': ['谢朓'],
             '王微': ['江淹'],
             '谢朓': ['江淹'],
             '鲍照': ['沈约'],
             '颜延之': ['谢超宗', '丘灵鞠', '刘祥', '檀超', '钟宪', '颜测', '顾则心'],
             '张融': ['孔稚珪']})

In [4]:
authors_set = set(shipin['Author'])
origin_set = set()
children_set = set()
for _, row in shipin.iterrows():
    author = row['Author']
    origin_str = row['Origin']
    if pd.notna(origin_str) and origin_str.strip():
        origins = [o.strip() for o in origin_str.split("、")]
        origin_set.update(origins)
        children_set.add(author)
roots = origin_set - children_set
print(roots)

{'《国风》', '张融', '《楚辞》', '《小雅》'}


In [5]:
def build_tree(node):
    return {
        "name": node + "*" if node in multiple_origin_flags else node,
        "multiple": node in multiple_origin_flags,
        "children": [build_tree(child) for child in graph.get(node, [])]
    }
forest = [build_tree(root) for root in roots]
forest

[{'name': '《国风》',
  'multiple': False,
  'children': [{'name': '《古诗》',
    'multiple': False,
    'children': [{'name': '刘桢',
      'multiple': False,
      'children': [{'name': '左思',
        'multiple': False,
        'children': [{'name': '陶潜*', 'multiple': True, 'children': []}]}]}]},
   {'name': '曹植',
    'multiple': False,
    'children': [{'name': '陆机',
      'multiple': False,
      'children': [{'name': '颜延之',
        'multiple': False,
        'children': [{'name': '谢超宗', 'multiple': False, 'children': []},
         {'name': '丘灵鞠', 'multiple': False, 'children': []},
         {'name': '刘祥', 'multiple': False, 'children': []},
         {'name': '檀超', 'multiple': False, 'children': []},
         {'name': '钟宪', 'multiple': False, 'children': []},
         {'name': '颜测', 'multiple': False, 'children': []},
         {'name': '顾则心', 'multiple': False, 'children': []}]}]},
     {'name': '谢灵运*', 'multiple': True, 'children': []}]}]},
 {'name': '张融',
  'multiple': False,
  'children':

In [6]:
def print_tree(tree, indent=0):
    prefix = "    " * indent
    label = tree["name"]
    print(f"{prefix}{label}")
    for child in tree["children"]:
        print_tree(child, indent + 1)

for tree in forest:
    print_tree(tree)

《国风》
    《古诗》
        刘桢
            左思
                陶潜*
    曹植
        陆机
            颜延之
                谢超宗
                丘灵鞠
                刘祥
                檀超
                钟宪
                颜测
                顾则心
        谢灵运*
张融
    孔稚珪
《楚辞》
    李陵
        班姬
        王粲
            潘岳
                郭璞
            张协
                谢灵运*
                鲍照*
                    沈约
            曹丕*
                嵇康
                应璩
                    陶潜*
            张华
                谢瞻
                谢混
                    谢朓
                        江淹*
                袁淑
                王微
                    江淹*
                王僧达
                鲍照*
                    沈约
            刘琨
            卢谌
        曹丕*
            嵇康
            应璩
                陶潜*
《小雅》
    阮籍
